In [1]:
# This notebook generates the results for table 1 (section 3.4)

import matplotlib.pyplot as plt
import numpy as np
from expected_cost import ec, utils
from expected_cost.data import create_scores_for_expts
from psrcal.losses import plot_reliability_diagram

from expected_cost.psrcal_wrappers import Brier, LogLoss, LogLossSE
outdir = "outputs/"
utils.mkdir_p(outdir)


In [4]:
num_targets = 2 # 2 for the results in the main paper (section 3), 10 for the appendix (section 5)

emp_epsr = {}
semiemp_epsr = {}

sep = ''  #'&'
endline = '' #'\\\\'

Ns = [200, 2000, 20000]

for N in Ns:

    # Create the scores using a general method from the repository and map the names to those
    # used in the paper. See notebook metric_comparison.ipynb for more details.
    feat_var = 0.15
    score_dict, targets = create_scores_for_expts(num_targets, N=N, P0=0.8, calibrate=True, simple_names=True, feat_var=feat_var, sim_name='gaussian_sim_md')

    for score_name in ['cal', 'mcp', 'mcs', 'mcps']:

        scores = score_dict[score_name]
        score_nameN = f'{score_name}-{N}'

        emp_epsr[score_nameN] = LogLoss(scores, targets)

        semiemp_epsr[score_nameN] = {}
        semiemp_epsr[score_nameN]['true'] = LogLossSE(scores, score_dict['cal'])
        for calt in ['aff', 'tem', 'his']:
            for cald in ['tt', 'xv']:
                caln = f'{calt}cal{cald}'
                semiemp_epsr[score_nameN][caln] = LogLossSE(scores, score_dict[score_name+'-'+caln])

print(f'Scores               ', end='')
for score_name in ['cal', 'mcs', 'mcp',  'mcps']:
    print(f'{sep} {score_name:21s} ', end='')
print(f'{endline}')

print(f'#Samples             ', end='')
for score_name in ['cal', 'mcs', 'mcp',  'mcps']:
    for N in Ns:
        score_nameN = f'{score_name}-{N}'
        print(f'{sep} {N:5d} ', end='')
print(f'{endline}')


for metric_name, metrics in semiemp_epsr['cal-200'].items():

    print(f'{sep}{metric_name:20s} ', end='')

    for score_name in ['cal', 'mcs', 'mcp',  'mcps']:
        for N in Ns:

            score_nameN = f'{score_name}-{N}'
            emp = emp_epsr[score_nameN]
            semiemp = semiemp_epsr[score_nameN][metric_name]
            ce_fit = int(np.round((emp-semiemp)/emp*100))
            print(f'{sep} {ce_fit:5d} ', end='')

    print(f'{endline}')



Scores                cal                    mcs                    mcp                    mcps                  
#Samples                200   2000  20000    200   2000  20000    200   2000  20000    200   2000  20000 
true                     -3     -5     -2     -7    -10     -5      4     -2      0      4     -2      0 
affcaltt                  0      0      0      0      0      0      0      0      0      0      0      0 
affcalxv                  1      0      0      1      0      0      1      0      0      0      0      0 
temcaltt                  1     -1      0      1     -1      0     29     28     28     29     28     28 
temcalxv                  1     -1      0      2     -1      0     29     28     28     29     28     28 
hiscaltt                 -6     -8     -9   -153   -143   -132    -19    -21    -17    -62    -84    -82 
hiscalxv                 -2     -8     -9   -171   -144   -132    -15    -21    -17    -62    -83    -82 
